In [1]:
from model import river_model, losses, dot_prod_attention
from data import data_generation, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks, metrics
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from inference import infer
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras
plt.style.use('ggplot')

Using TensorFlow backend.


In [3]:
import pandas as pd

In [4]:
attributes_numeric = pd.read_csv('/Users/omernivron/Downloads/att_numeric')

In [ ]:
attributes_numeric.iloc[:, 1:] = (attributes_numeric.iloc[:, 1:] - np.mean(attributes_numeric.iloc[:, 1:], axis = 0)) / np.std(attributes_numeric.iloc[:, 1:], axis = 0)

In [5]:
arr_pp = np.load('/Users/omernivron/Downloads/river_processed.npy')

In [6]:
arr_pe = np.load('/Users/omernivron/Downloads/river_processed_te.npy')

In [7]:
save_dir = '/Users/omernivron/Downloads/GPT_river'

In [26]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
m_tr = tf.keras.metrics.Mean()
m_te = tf.keras.metrics.Mean()

In [27]:
@tf.function
def train_step(decoder, optimizer_c, train_loss, m_tr, token_pos, time_pos, time_pos2, pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_log_sig = decoder(token_pos, time_pos, time_pos2, pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         tf.print(pred_sig)

        loss, mse, mask = losses.loss_function(tar_real, pred, pred_log_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
    m_tr.update_state(mse, mask)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))
    return tar_inp, tar_real, pred, pred_log_sig, mask

In [28]:
@tf.function
def test_step(decoder, test_loss, m_te, token_pos_te, time_pos_te, time_pos2_te, pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred_te, pred_log_sig_te = decoder(token_pos_te, time_pos_te, time_pos2_te, pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
    t_loss, t_mse, t_mask = losses.loss_function(tar_real_te, pred_te, pred_log_sig_te)
    test_loss(t_loss)
    m_te.update_state(t_mse, t_mask)
    return tar_real_te, pred_te, pred_log_sig_te, t_mask

In [33]:
tf.keras.backend.set_floatx('float64')

In [34]:
t1_tr = arr_pp[1::5]; t2_tr = arr_pp[2::5]
tar_tr = arr_pp[0::5];
token_tr = arr_pp[3::5]; basin_l_tr = arr_pp[4::5] 

In [35]:
t1_te = arr_pe[1::5]; t2_te = arr_pe[2::5]
tar_te = arr_pe[0::5];
token_te = arr_pe[3::5]; basin_l_te = arr_pe[4::5] 

In [36]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam(0.0004)
    decoder = river_model.Decoder(16)
    EPOCHS = 500
    batch_s  = 64
    run = 0; step = 0
    num_batches = int(tar_tr.shape[0] / batch_s)
    tf.random.set_seed(1)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer = optimizer_c, net = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT_river/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)
    #https://www.tensorflow.org/guide/checkpoint
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                m_tr.reset_states(); train_loss.reset_states()
                m_te.reset_states(); test_loss.reset_states()
                batch_tok_pos_tr, batch_tim_pos_tr, batch_tim_pos_tr2, batch_pos_tr, batch_tar_tr, _ = batch_creator.create_batch_river(token_tr, t1_tr, t2_tr, basin_l_tr,  attributes_numeric, tar_tr, batch_s=64)
                batch_pos_mask = masks.position_mask(batch_tok_pos_tr)
                tar_inp, tar_real, pred, pred_log_sig, mask = train_step(decoder, optimizer_c, train_loss, m_tr, batch_tok_pos_tr, batch_tim_pos_tr, batch_tim_pos_tr2, batch_pos_tr, batch_tar_tr, batch_pos_mask)

                if batch_n % 100 == 0:
                    batch_tok_pos_te, batch_tim_pos_te, batch_tim_pos_te2, batch_pos_te, batch_tar_te, _ = batch_creator.create_batch_river(token_te, t1_te, t2_te, basin_l_te,  attributes_numeric, tar_te, batch_s=64)
                    batch_pos_mask_te = masks.position_mask(batch_tok_pos_te)
                    tar_real_te, pred_te, pred_log_sig_te, t_mask = test_step(decoder, test_loss, m_te, batch_tok_pos_te, batch_tim_pos_te, batch_tim_pos_te2, batch_pos_te, batch_tar_te, batch_pos_mask_te)
                    helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), m_tr.result())
                    helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), m_tr.result(), m_te.result())
                    manager.save()
                step += 1
                ckpt.step.assign_add(1)

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Already exists
Restored from /Users/omernivron/Downloads/GPT_river/ckpt/check_0/ckpt-2118
Epoch 0 batch 0 train Loss 6.0632 test Loss 6.0758 with MSE metric 33784.0000
Epoch 0 batch 100 train Loss 6.3073 test Loss 6.1630 with MSE metric 54549.8594
Time taken for 1 epoch: 65.35824990272522 secs

Epoch 1 batch 0 train Loss 6.1756 test Loss 6.2554 with MSE metric 42485.8984
Epoch 1 batch 100 train Loss 6.2090 test Loss 6.2305 with MSE metric 44969.4570
Time taken for 1 epoch: 67.03810524940491 secs

Epoch 2 batch 0 train Loss 6.1176 test Loss 6.3076 with MSE metric 37789.6250
Epoch 2 batch 100 train Loss 6.0772 test Loss 6.2848 with MSE metric 33237.1719


KeyboardInterrupt: 

In [40]:
context_channels2 = ['prcp(mm/day)', 
'srad(W/m2)',  
'tmax(C)',
'tmin(C)', 
'vp(Pa)'] 

In [43]:
df.loc[:, context_channels2] = (df.loc[:, context_channels2] - np.mean(df.loc[:, context_channels2], axis = 0)) / np.std(df.loc[:, context_channels2], axis = 0) 

In [44]:
    selected_basins = pd.read_csv('/Users/omernivron/Downloads/basin_list.txt', header=None)


In [45]:
    test_basins = df.basin.unique()[~np.isin(df.basin.unique(), selected_basins)]


In [46]:
    list_to_drop = ['MNTH', 'DY', 'hru02', 'hru04', 'RAIM', 'TAIR', 'PET', 'ET', 'SWE', 'swe(mm)', 'PRCP', 'seed', 'id_lag', 'HR', 'dayl(s)', 'YR', 'MOD_RUN', 'id', 'DOY', 'DATE']
    df.drop(columns= list_to_drop, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df[df['OBS_RUN'] >= 0]
    cols = df.columns.to_list()
    cols = [cols[5]] + cols[:5] + cols[6:]
    df = df[cols]

In [47]:
pd.DataFrame.to_csv(df, 'river_flow_processed.csv')

KeyboardInterrupt: 